In [16]:
import pandas as pd

In [23]:
import mysql.connector

In [36]:
#input the credentials
connection = mysql.connector.connect(
    host="127.0.0.1",  # Hostname
    user="dbukka",   # Username
    password="OAPHLSH[zWRAaO*I",  # Password
    database="dbukka_db"  # Database name
)

cursor = connection.cursor()

In [7]:
# Step 2: Read the cleaned CSV file
csv_file_path = "enhanced_stroke_data.csv"  # Update with the correct path to your CSV file
data = pd.read_csv(csv_file_path)
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,metric_id,lifestyle_id,condition_id,location_id
0,9046,Male,67,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1,1,1,1,1
1,51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1,2,2,2,2
2,31112,Male,80,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,3,3,3,3
3,60182,Female,49,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1,4,4,4,4
4,1665,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1,5,5,5,5


In [28]:
# Step 3: Populate Patient Table
print("Inserting data into Patient table...")
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Patient (Patient_ID, Gender, Age)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE Gender=Gender;
    """, (index + 1, row['gender'], row['age']))

# Step 4: Populate Conditions Table
print("Inserting data into Conditions table...")
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Conditions (Conditions_ID, Patient_ID, Hypertension, Heart_Disease, Stroke)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE Stroke=Stroke;
    """, (index + 101, index + 1, row['hypertension'], row['heart_disease'], row['stroke']))

# Step 5: Populate Lifestyle Table
print("Inserting data into Lifestyle table...")
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Lifestyle (Lifestyle_ID, Patient_ID, Conditions_ID, Smoking_Status, Work_Type, Ever_Married)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE Work_Type=Work_Type;
    """, (index + 201, index + 1, index + 101, row['smoking_status'], row['work_type'], row['ever_married']))

# Step 6: Populate Metrics Table
print("Inserting data into Metrics table...")
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Metrics (Metrics_ID, Patient_ID, Conditions_ID, Avg_Glucose_Level, BMI)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE BMI=BMI;
    """, (index + 301, index + 1, index + 101, row['avg_glucose_level'], row['bmi']))

# Step 7: Populate Location Table
print("Inserting data into Location table...")
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Location (Location_ID, Patient_ID, Conditions_ID, Region)
        VALUES (%s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE Region=Region;
    """, (index + 401, index + 1, index + 101, row['Residence_type']))

# Commit and close
connection.commit()
print("All data inserted successfully!")

cursor.close()
connection.close()

Inserting data into Patient table...
Inserting data into Conditions table...
Inserting data into Lifestyle table...
Inserting data into Metrics table...
Inserting data into Location table...
All data inserted successfully!


# Objective 1: Identify Key Health Factors Influencing Stroke

## Query 1.1: Patients with Stroke, Hypertension, and High BMI

In [37]:
# Patients with Stroke, Hypertension, and High BMI
cursor.execute("""
SELECT p.Patient_ID, p.Age, m.BMI, m.Avg_Glucose_Level, 
       c.Hypertension, c.Heart_Disease, c.Stroke
FROM Patient p
JOIN Conditions c ON p.Patient_ID = c.Patient_ID
JOIN Metrics m ON c.Conditions_ID = m.Conditions_ID
WHERE c.Stroke = 1 AND c.Hypertension = 1 AND m.BMI > 30
ORDER BY m.BMI DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)

(22, 52, 48.9, 233.29, 1, 0, 1)
(164, 78, 45.7, 203.87, 1, 0, 1)
(197, 70, 45.5, 242.52, 1, 0, 1)
(31, 57, 44.2, 212.08, 1, 0, 1)
(192, 51, 42.5, 112.16, 1, 0, 1)
(244, 68, 40.5, 247.51, 1, 1, 1)
(218, 55, 40.0, 210.4, 1, 1, 1)
(50, 39, 39.2, 58.09, 1, 0, 1)
(185, 61, 37.4, 112.24, 1, 1, 1)
(176, 72, 37.1, 185.49, 1, 0, 1)
(144, 69, 36.8, 72.17, 1, 1, 1)
(133, 69, 36.5, 182.99, 1, 0, 1)
(131, 73, 36.5, 190.14, 1, 0, 1)
(124, 56, 35.8, 249.31, 1, 0, 1)
(230, 80, 35.4, 66.03, 1, 0, 1)
(165, 70, 34.2, 89.13, 1, 0, 1)
(209, 78, 33.6, 134.8, 1, 0, 1)
(46, 76, 33.6, 243.58, 1, 0, 1)
(149, 57, 32.9, 67.41, 1, 0, 1)
(174, 80, 32.9, 91.02, 1, 0, 1)
(59, 73, 32.8, 194.99, 1, 0, 1)
(202, 80, 31.5, 175.29, 1, 1, 1)
(54, 77, 31.4, 124.13, 1, 0, 1)
(194, 68, 31.1, 271.74, 1, 1, 1)
(73, 66, 31.1, 116.55, 1, 0, 1)
(116, 71, 30.9, 216.94, 1, 1, 1)
(16, 50, 30.9, 167.41, 1, 0, 1)
(122, 53, 30.3, 62.55, 1, 0, 1)
(107, 50, 30.3, 73.18, 1, 0, 1)


## Query 1.2: Average Glucose and BMI for Stroke vs Non-Stroke Patients

In [38]:
# Average Glucose and BMI for Stroke vs Non-Stroke Patients

cursor.execute("""
SELECT c.Stroke, 
       ROUND(AVG(m.Avg_Glucose_Level), 2) AS Avg_Glucose,
       ROUND(AVG(m.BMI), 2) AS Avg_BMI
FROM Conditions c
JOIN Metrics m ON c.Conditions_ID = m.Conditions_ID
GROUP BY c.Stroke;
""")

records = cursor.fetchall()
for record in records:
    print(record)


(0, 104.8, 28.83)
(1, 132.54, 30.22)


## Interpretation 
1. Stroke Grouping:
Stroke = 0: Represents non-stroke patients.
Stroke = 1: Represents stroke patients.
2. Average Glucose Levels:
Non-Stroke Patients: 104.8

This is within the normal glucose range (typically 70-140 mg/dL).
Indicates non-stroke patients generally have healthy glucose levels.
Stroke Patients: 132.54

This is significantly higher than for non-stroke patients.
Interpretation: Elevated glucose levels (hyperglycemia) are strongly associated with stroke risk.
Practical Insight: Managing glucose levels can help reduce the risk of stroke.
3. Average BMI:
Non-Stroke Patients: 28.83

This falls in the overweight category (BMI between 25 and 29.9).
Indicates a slightly higher BMI but not yet obesity.
Stroke Patients: 30.22

This is in the obese category (BMI ≥ 30).
Interpretation: Obesity is a contributing factor to stroke risk.
Practical Insight: BMI management (weight control) is important in reducing stroke incidence.
Key Findings for Objective 1:
Stroke patients have higher average glucose levels (132.54 vs 104.8).

Hyperglycemia may play a significant role in increasing stroke risk.
Stroke patients also have a higher BMI (30.22 vs 28.83).

Although the difference is small, it confirms that obesity is more common in stroke patients.
Conclusion:
Both hyperglycemia and obesity are contributing factors to stroke risk.
Practical interventions:
Monitor and manage glucose levels for at-risk patients.
Promote weight management strategies to reduce BMI.

In [41]:
# Difference (or delta) between stroke and non-stroke groups for average glucose and BMI
cursor.execute("""
SELECT 
    'Glucose' AS Metric,
    ROUND(AVG(CASE WHEN c.Stroke = 1 THEN m.Avg_Glucose_Level END) 
        - AVG(CASE WHEN c.Stroke = 0 THEN m.Avg_Glucose_Level END), 2) AS Difference
FROM Conditions c
JOIN Metrics m ON c.Conditions_ID = m.Conditions_ID

UNION ALL

SELECT 
    'BMI' AS Metric,
    ROUND(AVG(CASE WHEN c.Stroke = 1 THEN m.BMI END) 
        - AVG(CASE WHEN c.Stroke = 0 THEN m.BMI END), 2) AS Difference
FROM Conditions c
JOIN Metrics m ON c.Conditions_ID = m.Conditions_ID;
""")

records = cursor.fetchall()
for record in records:
    print(record)


('Glucose', 27.75)
('BMI', 1.39)


## Interpretation:
Glucose Difference (27.75):

Stroke patients, on average, have a 27.75 units higher glucose level than non-stroke patients.
This indicates a strong correlation between elevated glucose levels (hyperglycemia) and stroke risk.
Practical Insight: Patients with higher average glucose levels might be at increased risk of stroke, highlighting the need for glucose management in stroke prevention.
BMI Difference (1.39):

Stroke patients, on average, have a slightly higher BMI (by 1.39 units) than non-stroke patients.
This suggests that while BMI is a contributing factor, its impact might be less significant compared to glucose levels.
Practical Insight: Obesity (high BMI) may still play a role in stroke risk, but other factors like glucose levels or hypertension might be more impactful.

## Objective 2: Analyze the Impact of Lifestyle on Stroke Risk

# Query 2.1: Stroke Count Grouped by Smoking Status

In [42]:
# Stroke Count Grouped by Smoking Status
cursor.execute("""
SELECT l.Smoking_Status, COUNT(c.Stroke) AS Stroke_Count
FROM Lifestyle l
JOIN Conditions c ON l.Conditions_ID = c.Conditions_ID
WHERE c.Stroke = 1
GROUP BY l.Smoking_Status
ORDER BY Stroke_Count DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)


('never smoked', 90)
('formerly smoked', 70)
('Unknown', 47)
('smokes', 42)


## Interpretation
1. Smoking Status and Stroke Risk
Never Smoked (90 cases):

The highest number of stroke cases are observed among individuals who never smoked.
Possible Explanation:
Non-smoking patients may still have other contributing risk factors (e.g., hypertension, high glucose levels, obesity, heart disease).
It also highlights that smoking is not the only factor influencing stroke risk.
Formerly Smoked (70 cases):

Patients who quit smoking still show a relatively high stroke count.
Interpretation:
Previous smoking exposure may have long-term effects on cardiovascular health, increasing stroke risk even after quitting.
Unknown (47 cases):

The smoking status for these patients is unknown.
Impact:
This incomplete data may mask patterns.
Suggestion: Further data collection or cleaning to clarify smoking history could improve the analysis.
Smokes (42 cases):

Surprisingly, active smokers have the lowest stroke count among the groups.
Possible Explanation:
Smaller sample size for smokers in your dataset.
Smokers might be underrepresented compared to non-smokers.
Alternatively, smoking could correlate with earlier mortality from other diseases before stroke develops.
Key Insights:
Stroke occurs most frequently among people who never smoked and those who formerly smoked.
Smoking Status Alone is not sufficient to determine stroke risk; other factors like hypertension, BMI, and glucose levels must be analyzed alongside.
The "Unknown" group highlights a need for better data completeness to confirm smoking's role in stroke outcomes.


# Next Steps:
Cross-check smoking status with other risk factors (e.g., hypertension, glucose levels):

Do non-smokers have higher hypertension rates?
Are formerly smokers more obese? 

In [45]:
# Do non-smokers have higher hypertension rates?
cursor.execute("""
SELECT 
    l.Smoking_Status AS 'Smoking Status', 
    COUNT(CASE WHEN c.Hypertension = 1 THEN 1 END) AS 'Hypertensive Count',
    COUNT(*) AS 'Total Count',
    ROUND((COUNT(CASE WHEN c.Hypertension = 1 THEN 1 END) / COUNT(*)) * 100, 2) AS 'Hypertension Percentage'
FROM Lifestyle l
JOIN Conditions c ON l.Conditions_ID = c.Conditions_ID
GROUP BY l.Smoking_Status
ORDER BY 'Hypertension Percentage' DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)

('formerly smoked', 120, 885, Decimal('13.56'))
('never smoked', 232, 1892, Decimal('12.26'))
('smokes', 94, 789, Decimal('11.91'))
('Unknown', 52, 1544, Decimal('3.37'))


In [ ]:
## Interpretation
1. Formerly Smoked (13.56%):
Patients who quit smoking have the highest percentage of hypertension (13.56%).
Insight: Long-term effects of past smoking could contribute to higher hypertension rates even after quitting.
2. Never Smoked (12.26%):
Surprisingly, patients who never smoked also have a notable hypertension rate (12.26%).
Insight: Other risk factors (e.g., obesity, glucose levels, genetics) likely contribute to hypertension in non-smokers.
3. Smokes (11.91%):
Current smokers have a slightly lower hypertension rate (11.91%) compared to former and never smokers.
Insight: This may be due to sample size bias or smokers experiencing other health complications before hypertension develops.
4. Unknown (3.37%):
Patients with an unknown smoking status have the lowest hypertension rate (3.37%).
Insight: Incomplete or misreported data may affect this result. These cases need further clarification.
Key Takeaways:
Former Smokers have the highest hypertension rate, indicating lingering effects of smoking on cardiovascular health.
Non-Smokers also show notable hypertension rates, suggesting other contributing factors like diet, genetics, or stress.
Further analysis can include cross-checking hypertension with BMI or glucose levels to identify additional correlations.

In [46]:
# Are formerly smokers more obese? 
cursor.execute("""
SELECT 
    l.Smoking_Status AS 'Smoking Status', 
    ROUND(AVG(m.BMI), 2) AS 'Average BMI'
FROM Lifestyle l
JOIN Conditions c ON l.Conditions_ID = c.Conditions_ID
JOIN Metrics m ON c.Conditions_ID = m.Conditions_ID
GROUP BY l.Smoking_Status
ORDER BY 'Average BMI' DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)

('formerly smoked', 30.65)
('never smoked', 29.96)
('smokes', 30.43)
('Unknown', 25.79)


## Query 2.2: Stroke Count Grouped by Work Type

In [ ]:
# Stroke Count Grouped by Work Type

In [44]:
# Stroke Count Grouped by Smoking Status
cursor.execute("""
SELECT l.Work_Type, COUNT(c.Stroke) AS Stroke_Count
FROM Lifestyle l
JOIN Conditions c ON l.Conditions_ID = c.Conditions_ID
WHERE c.Stroke = 1
GROUP BY l.Work_Type
ORDER BY Stroke_Count DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)

('Private', 149)
('Self-employed', 65)
('Govt_job', 33)
('children', 2)


## Interpretation:
Private Sector (149 cases):

The highest number of stroke cases are observed among individuals working in the private sector.
This could be due to higher stress, sedentary lifestyles, or other factors common in private-sector jobs.
Self-Employed (65 cases):

Stroke cases are also significant among self-employed individuals, indicating possible work-related stress or inconsistent access to healthcare.
Government Jobs (33 cases):

Fewer stroke cases are seen among individuals with government jobs.
This may suggest better job security, healthcare access, or a balanced lifestyle.
Children (2 cases):

Very few stroke cases are observed in the children category, as expected.
Stroke is rare in children compared to adults.
Key Takeaways:
The Private sector has the most stroke cases, suggesting a need to address work-related health risks.
Self-employed individuals may also benefit from targeted health interventions.
Government workers and children have comparatively lower stroke incidences.

# Objective 3: Geographical Insights into Health Conditions and Stroke

In [ ]:
## Query 3.1: Stroke Cases by Region

In [47]:
# Stroke Cases by Region

cursor.execute("""
SELECT loc.Region, COUNT(c.Stroke) AS Stroke_Count
FROM Location loc
JOIN Conditions c ON loc.Conditions_ID = c.Conditions_ID
WHERE c.Stroke = 1
GROUP BY loc.Region
ORDER BY Stroke_Count DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)


('Urban', 135)
('Rural', 114)


# Query 3.2: Region-Wise Distribution of Patients with Hypertension

In [48]:
# Region-Wise Distribution of Patients with Hypertension

cursor.execute("""
SELECT loc.Region, COUNT(c.Hypertension) AS Hypertension_Count
FROM Location loc
JOIN Conditions c ON loc.Conditions_ID = c.Conditions_ID
WHERE c.Hypertension = 1
GROUP BY loc.Region
ORDER BY Hypertension_Count DESC;
""")

records = cursor.fetchall()
for record in records:
    print(record)


('Rural', 251)
('Urban', 247)
